In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 3, Finished, Available, Finished)

In [2]:
# Loading Student data

df_students = spark.sql("""

SELECT
    p.StudentNumber,
    p.Age,
    p.IsChurchMemberCMIS,
    p.CountryName,
    h.mshied_academicperiodidname AS AcademicPeriod,
    DATEDIFF(CURRENT_DATE(),c.cmc_lastcompletedactivitydate) AS lastcompletedactivitydate,
    DATEDIFF(CURRENT_DATE(),s.pw_canvaslastlogin) AS canvaslastlogin,
    COUNT(CASE WHEN h.mshied_registrationstatusidname IN ('Grade Posted') THEN 1 END) AS Num_Courses,
    SUM(CASE WHEN h.mshied_registrationstatusidname = 'Dropped' THEN 1 ELSE 0 END) AS Dropped_Courses,
    AVG(CASE WHEN h.mshied_registrationstatusidname IN ('Grade Posted','Current') THEN h.pw_percentagegrade END) AS avg_percentage_score,
    SUM(CASE WHEN h.pw_percentagegrade < 80 THEN 1 ELSE 0 END) AS courses_under_80
FROM
    dbo.contact c
    LEFT JOIN dbo.person AS p ON c.cmc_studentnumber = p.StudentNumber
    LEFT JOIN dbo.mshied_coursehistory h ON c.contactid = h.mshied_studentid
    LEFT JOIN dbo.pw_student AS s ON c.contactid = s.pw_contact
WHERE 
    h.mshied_academicperiodidname IN (
        '2025 Term 1',
        '2025 Term 2',
        '2025 Term 3',
        '2025 Term 4')
    AND mshied_academicperiodidname is not null
    AND h.mshied_academicperiodidname != 'Transfer Term'
    AND mshied_registrationstatusidname != 'Future'
    AND h.pw_percentagegrade is not null   
GROUP BY
    p.StudentNumber, h.mshied_academicperiodidname, p.Age, p.IsChurchMemberCMIS, p.CountryName, c.cmc_lastcompletedactivitydate, s.pw_canvaslastlogin
HAVING AVG(CASE WHEN h.mshied_registrationstatusidname IN ('Grade Posted','Current') 
                THEN h.pw_percentagegrade END) IS NOT NULL
""")

df_students = df_students.toPandas()


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 4, Finished, Available, Finished)

In [3]:
# Loading HDI index by Country table

df_hdi = pd.read_csv("/lakehouse/default/Files/HDR25_Statistical_Annex_HDI_Table.csv", sep = ';')
df_hdi.tail()


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 5, Finished, Available, Finished)

,CountryName,HDI rank,Human Development Index (HDI) 2023
211,Macau,47,"0,909"
212,Puerto Rico,63,"0,868"
213,curacao,93,"0,811"
214,Sint Maarten,133,"0,702"
215,Russia,64,"0,832"


In [4]:
# because the , on the column 2, it need to replace ',' to '.' and then change type to float

df_hdi['Human Development Index (HDI) 2023'] = (df_hdi['Human Development Index (HDI) 2023'].str.replace(',','.').astype(float))
df_hdi.dtypes

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 6, Finished, Available, Finished)

CountryName                            object
HDI rank                                int64
Human Development Index (HDI) 2023    float64
dtype: object

In [5]:
df_students['CountryName'] = df_students['CountryName'].str.strip().str.lower()
df_hdi['CountryName'] = df_hdi['CountryName'].str.strip().str.lower()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 7, Finished, Available, Finished)

In [6]:
# Formating some countries that has special characters
df_students['CountryName'] = df_students['CountryName'].str.replace('[.,]', '', regex=True)

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 8, Finished, Available, Finished)

Adding HDI to the student data

In [7]:
df = df_students.merge(df_hdi[['CountryName', 'HDI rank','Human Development Index (HDI) 2023']], on='CountryName', how='left')

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 9, Finished, Available, Finished)

In [8]:
# checking how many records are

df.shape

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 10, Finished, Available, Finished)

(113670, 13)

In [9]:
# checking columns

df.columns

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 11, Finished, Available, Finished)

Index(['StudentNumber', 'Age', 'IsChurchMemberCMIS', 'CountryName',
       'AcademicPeriod', 'lastcompletedactivitydate', 'canvaslastlogin',
       'Num_Courses', 'Dropped_Courses', 'avg_percentage_score',
       'courses_under_80', 'HDI rank', 'Human Development Index (HDI) 2023'],
      dtype='object')

In [10]:
# checking types

df.dtypes

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 12, Finished, Available, Finished)

StudentNumber                           int32
Age                                    object
IsChurchMemberCMIS                     object
CountryName                            object
AcademicPeriod                         object
lastcompletedactivitydate             float64
canvaslastlogin                       float64
Num_Courses                             int64
Dropped_Courses                         int64
avg_percentage_score                   object
courses_under_80                        int64
HDI rank                              float64
Human Development Index (HDI) 2023    float64
dtype: object

In [11]:
df.head()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 13, Finished, Available, Finished)

,StudentNumber,Age,IsChurchMemberCMIS,CountryName,AcademicPeriod,lastcompletedactivitydate,canvaslastlogin,Num_Courses,Dropped_Courses,avg_percentage_score,courses_under_80,HDI rank,Human Development Index (HDI) 2023
0,30018684,23,True,united states,2025 Term 2,139.0,136.0,2,0,96.165000,0,17.0,0.938
1,30016691,43,True,nigeria,2025 Term 2,111.0,81.0,2,0,24.810000,2,164.0,0.560
2,20058923,29,True,brazil,2025 Term 2,107.0,167.0,4,0,15.222500,4,84.0,0.786
3,30014362,50,True,united states,2025 Term 2,76.0,74.0,2,0,95.195000,0,17.0,0.938
4,22813048,23,True,brazil,2025 Term 3,138.0,72.0,1,0,75.950000,1,84.0,0.786


In [12]:
# checking null values
 
df.isnull().sum()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 14, Finished, Available, Finished)

StudentNumber                            0
Age                                      2
IsChurchMemberCMIS                     435
CountryName                           1208
AcademicPeriod                           0
lastcompletedactivitydate              563
canvaslastlogin                         15
Num_Courses                              0
Dropped_Courses                          0
avg_percentage_score                     0
courses_under_80                         0
HDI rank                              1212
Human Development Index (HDI) 2023    1212
dtype: int64

In [13]:
# Dropping records where country name is null (no HDI in concecuence)
df = df.dropna(subset=['CountryName','Age'])
df.isnull().sum()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 15, Finished, Available, Finished)

StudentNumber                           0
Age                                     0
IsChurchMemberCMIS                      0
CountryName                             0
AcademicPeriod                          0
lastcompletedactivitydate             561
canvaslastlogin                        12
Num_Courses                             0
Dropped_Courses                         0
avg_percentage_score                    0
courses_under_80                        0
HDI rank                                4
Human Development Index (HDI) 2023      4
dtype: int64

In [14]:
df[df['HDI rank'].isnull()]

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 16, Finished, Available, Finished)

,StudentNumber,Age,IsChurchMemberCMIS,CountryName,AcademicPeriod,lastcompletedactivitydate,canvaslastlogin,Num_Courses,Dropped_Courses,avg_percentage_score,courses_under_80,HDI rank,Human Development Index (HDI) 2023
22564,30036923,20,True,wallis and futuna islands,2025 Term 2,174.0,163.0,1,0,31.760000,1,NaN,NaN
40163,30027586,31,True,northern mariana islands,2025 Term 2,112.0,135.0,2,0,34.625000,2,NaN,NaN
55751,13081365,34,True,northern mariana islands,2025 Term 2,107.0,137.0,1,0,90.570000,0,NaN,NaN
86451,30018859,36,True,northern mariana islands,2025 Term 2,4.0,1.0,2,0,32.635000,2,NaN,NaN


In [15]:
# Dropping null on HDI Rank and canvaslastlogin column
df = df.dropna(subset=['HDI rank','canvaslastlogin'])

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 17, Finished, Available, Finished)

In [17]:
# Replacing NA on the lastcompletedactivitydate column with the mean
mean_col = df['lastcompletedactivitydate'].mean()
df['lastcompletedactivitydate'].fillna(mean_col, inplace = True)

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 19, Finished, Available, Finished)

In [18]:
# checking count of duplicates

df.duplicated(subset='StudentNumber', keep = False).sum()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 20, Finished, Available, Finished)

92075

In [19]:
# Changing columns types

cols_to_convert = ['Age','Num_Courses','Dropped_Courses','avg_percentage_score','courses_under_80','HDI rank','Human Development Index (HDI) 2023']
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
df.loc[:,'IsChurchMemberCMIS'] = df['IsChurchMemberCMIS'].astype(bool)
columns = ['CountryName']
for col in columns:
    df.loc[:,col] = df[col].astype(str)
df.dtypes

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 21, Finished, Available, Finished)

StudentNumber                           int32
Age                                     int64
IsChurchMemberCMIS                     object
CountryName                            object
AcademicPeriod                         object
lastcompletedactivitydate             float64
canvaslastlogin                       float64
Num_Courses                             int64
Dropped_Courses                         int64
avg_percentage_score                  float64
courses_under_80                        int64
HDI rank                              float64
Human Development Index (HDI) 2023    float64
dtype: object

In [20]:
df.nunique()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 22, Finished, Available, Finished)

StudentNumber                         56587
Age                                      74
IsChurchMemberCMIS                        2
CountryName                             170
AcademicPeriod                            3
lastcompletedactivitydate               243
canvaslastlogin                         192
Num_Courses                               9
Dropped_Courses                           5
avg_percentage_score                  24968
courses_under_80                          7
HDI rank                                134
Human Development Index (HDI) 2023      138
dtype: int64

In [21]:
# basic info

df.describe()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 23, Finished, Available, Finished)

,StudentNumber,Age,lastcompletedactivitydate,canvaslastlogin,Num_Courses,Dropped_Courses,avg_percentage_score,courses_under_80,HDI rank,Human Development Index (HDI) 2023
count,1.124440e+05,112444.000000,112444.000000,112444.000000,112444.000000,112444.000000,112444.000000,112444.000000,112444.000000,112444.000000
mean,2.384393e+07,31.800879,23.799705,18.645699,1.973240,0.032372,76.494572,0.611371,107.973151,0.709808
std,5.305972e+06,10.711378,39.173753,32.546082,0.839781,0.205233,28.433073,0.850106,60.938373,0.159746
min,1.700000e+01,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.388000
25%,2.228097e+07,23.000000,4.000000,1.000000,1.000000,0.000000,68.927500,0.000000,29.000000,0.560000
50%,2.304183e+07,29.000000,5.000000,2.000000,2.000000,0.000000,89.480000,0.000000,138.000000,0.649000
75%,3.001840e+07,38.000000,20.000000,18.000000,2.000000,0.000000,95.656667,1.000000,164.000000,0.906000
max,3.006870e+07,125.000000,350.000000,254.000000,8.000000,4.000000,200.000000,6.000000,193.000000,0.983000


In [22]:
# dropping outliers

df = df[(df['Age']> 16) & (df['Age'] < 85)] # age outliers
df = df[(df['avg_percentage_score']<= 100)] # score outliers
df.describe()


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 24, Finished, Available, Finished)

,StudentNumber,Age,lastcompletedactivitydate,canvaslastlogin,Num_Courses,Dropped_Courses,avg_percentage_score,courses_under_80,HDI rank,Human Development Index (HDI) 2023
count,1.119590e+05,111959.000000,111959.000000,111959.000000,111959.000000,111959.000000,111959.000000,111959.000000,111959.000000,111959.000000
mean,2.383531e+07,31.810011,23.836851,18.682143,1.973553,0.032405,76.414922,0.613242,107.980019,0.709784
std,5.303729e+06,10.681220,39.209010,32.579749,0.840346,0.205379,28.432750,0.850874,60.938031,0.159754
min,1.700000e+01,17.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.388000
25%,2.228080e+07,23.000000,4.000000,1.000000,1.000000,0.000000,68.790000,0.000000,29.000000,0.560000
50%,2.304141e+07,29.000000,5.000000,2.000000,2.000000,0.000000,89.430000,0.000000,138.000000,0.649000
75%,3.001827e+07,38.000000,20.000000,18.000000,2.000000,0.000000,95.610000,1.000000,164.000000,0.906000
max,3.006870e+07,83.000000,350.000000,254.000000,8.000000,4.000000,100.000000,6.000000,193.000000,0.983000


Labels

In [23]:
# Create a table to compare if the student was enrolled next semester comparing terms

term = pd.read_csv("/lakehouse/default/Files/Terms_csv.csv", sep = ';')
term.tail()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 25, Finished, Available, Finished)

,rank,Termname
0,14,2025 Term 1
1,15,2025 Term 2
2,16,2025 Term 3
3,17,2025 Term 4
4,18,2025 Term 5


In [24]:
# Normalizar los nombres de AcademicPeriod en df y Termname en term
df["AcademicPeriod"] = df["AcademicPeriod"].str.strip().str.lower()
term["Termname"] = term["Termname"].str.strip().str.lower()


# Paso 1: Mapeamos el ranking a df
df = df.merge(
    term.rename(columns={"Termname": "AcademicPeriod", "rank": "term_rank"}),
    on="AcademicPeriod",
    how="left"
)

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 26, Finished, Available, Finished)

In [25]:
# Paso 2: Crear una columna booleana is_enrolled_next_term
# Para eso vamos a:
# - Agrupar por StudentNumber
# - Ver si hay un rank siguiente

# Primero, por claridad, ordenamos por StudentNumber y term_rank
df = df.sort_values(["StudentNumber", "term_rank"])

# Inicializamos la columna en False
df["is_enrolled_next_term"] = False

# Paso 3: Chequear si hay term_rank siguiente
# Vamos a usar shift(-1) dentro de cada StudentNumber
df["next_term_rank"] = df.groupby("StudentNumber")["term_rank"].shift(-1)

# Si el siguiente rank es exactamente +1, entonces True
df["is_enrolled_next_term"] = df["next_term_rank"] == (df["term_rank"] + 1)

# Si quieres, puedes eliminar la columna auxiliar:
df = df.drop(columns=["next_term_rank"])

# Mostrar resultado
print(df[["AcademicPeriod", "term_rank", "is_enrolled_next_term"]])


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 27, Finished, Available, Finished)

       AcademicPeriod  term_rank  is_enrolled_next_term
63772     2025 term 2         15                   True
105590    2025 term 3         16                   True
15816     2025 term 4         17                  False
41490     2025 term 2         15                   True
48814     2025 term 3         16                   True
...               ...        ...                    ...
54441     2025 term 4         17                  False
91582     2025 term 4         17                  False
39189     2025 term 2         15                   True
57810     2025 term 3         16                   True
74722     2025 term 4         17                  False

[111959 rows x 3 columns]


Features

In [26]:
# Convert all boolean columns to integers (1 for True, 0 for False)
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)
df['IsChurchMemberCMIS'] = (
    df['IsChurchMemberCMIS']
    .astype(str)
    .str.lower()
    .map({'true': 1, 'false': 0})
    .astype(int)
)


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 28, Finished, Available, Finished)

In [27]:
# drop columns and rows that won't be use it
df = df[df['AcademicPeriod'] != '2025 term 4']
df = df.drop(columns =['AcademicPeriod','CountryName','term_rank'])


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 29, Finished, Available, Finished)

In [28]:
df.head()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 30, Finished, Available, Finished)

,StudentNumber,Age,IsChurchMemberCMIS,lastcompletedactivitydate,canvaslastlogin,Num_Courses,Dropped_Courses,avg_percentage_score,courses_under_80,HDI rank,Human Development Index (HDI) 2023,is_enrolled_next_term
63772,17,28,1,139.0,42.0,2,0,96.875000,0,47.0,0.865,1
105590,17,28,1,139.0,42.0,1,0,100.000000,0,47.0,0.865,1
41490,20,21,1,56.0,16.0,2,0,99.535000,0,16.0,0.939,1
48814,20,21,1,56.0,16.0,3,0,96.986667,0,16.0,0.939,1
78431,30,35,0,80.0,1.0,2,0,95.885000,0,143.0,0.628,1


In [29]:
# Savind as Delta
df_spark = spark.createDataFrame(df)

# Formating columns special characters
for col in df_spark.columns:
    new_col = col.strip().replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_")
    df_spark = df_spark.withColumnRenamed(col, new_col)

# Saving
path = "Tables/dbo/Student_variables_non_scaled"

df_spark.write.format("delta") \
    .mode("overwrite") \
    .save(path)

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 31, Finished, Available, Finished)

In [30]:
# Scaling numeric values

from sklearn.preprocessing import StandardScaler

cols_to_scale = cols_to_convert

scaler = StandardScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 32, Finished, Available, Finished)

In [31]:
df.head()

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 33, Finished, Available, Finished)

,StudentNumber,Age,IsChurchMemberCMIS,lastcompletedactivitydate,canvaslastlogin,Num_Courses,Dropped_Courses,avg_percentage_score,courses_under_80,HDI rank,Human Development Index (HDI) 2023,is_enrolled_next_term
63772,17,-0.367009,1,139.0,42.0,0.023010,-0.155645,0.709735,-0.711054,-0.960583,0.933934,1
105590,17,-0.367009,1,139.0,42.0,-1.162215,-0.155645,0.822996,-0.711054,-0.960583,0.933934,1
41490,20,-1.018409,1,56.0,16.0,0.023010,-0.155645,0.806143,-0.711054,-1.464593,1.393431,1
48814,20,-1.018409,1,56.0,16.0,1.208236,-0.155645,0.713782,-0.711054,-1.464593,1.393431,1
78431,30,0.284390,0,80.0,1.0,0.023010,-0.155645,0.673853,-0.711054,0.600222,-0.537697,1


In [32]:
df.shape

StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 34, Finished, Available, Finished)

(74945, 12)

In [33]:
cols_con_nulos = df.columns[df.isnull().any()]
print(cols_con_nulos)


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 35, Finished, Available, Finished)

Index([], dtype='object')


In [34]:
df[df['avg_percentage_score'].isnull()]


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 36, Finished, Available, Finished)

,StudentNumber,Age,IsChurchMemberCMIS,lastcompletedactivitydate,canvaslastlogin,Num_Courses,Dropped_Courses,avg_percentage_score,courses_under_80,HDI rank,Human Development Index (HDI) 2023,is_enrolled_next_term


Saving dataset for later spltting and modeling

In [35]:
# Savind as Delta
df_spark = spark.createDataFrame(df)

# Formating columns special characters
for col in df_spark.columns:
    new_col = col.strip().replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_")
    df_spark = df_spark.withColumnRenamed(col, new_col)

# Saving
path = "Tables/dbo/Student_variables"

df_spark.write.format("delta") \
    .mode("overwrite") \
    .save(path)


StatementMeta(, e909c68c-fa5a-447e-8caa-21637194dc01, 37, Finished, Available, Finished)